<h1 align='left'>  MLFlow Example - Gautam Kumar - Data Scientist II, Ex-AWS, Vancouver </h1>

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [44]:
# Here I am Creating an skewed binary classification dataset
X, y = make_classification(n_samples=3500, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.85, 0.15], flip_y=0, random_state=21)

np.unique(y, return_counts=True)

(array([0, 1]), array([2975,  525]))

In [65]:
# Below code splits the the dataset to training and testing data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=21)

print("Shape of X_traing ------->>>>>>>>. X_traing.shape  ", X_train.shape)
print("Shape of Y_traing ------->>>>>>>>. Y_traing.shape  ", y_train.shape)
print("Shape of X_test ------->>>>>>>>. X_test.shape  ", X_test.shape)
print("Shape of Y_test ------->>>>>>>>. Y_test.shape  ", y_test.shape)

Shape of X_traing ------->>>>>>>>. X_traing.shape   (2625, 10)
Shape of Y_traing ------->>>>>>>>. Y_traing.shape   (2625,)
Shape of X_test ------->>>>>>>>. X_test.shape   (875, 10)
Shape of Y_test ------->>>>>>>>. Y_test.shape   (875,)


## Exp 1: Classification by training Logistic Regression 

In [66]:
logistic_regr = LogisticRegression(C=1, solver='liblinear')
logistic_regr.fit(x_train, y_train)
y_pred_log_reg = logistic_regr.predict(x_test)
print("GK Classification report for Logistic Regression:" + classification_report(y_test, y_pred_log_reg))

GK Classification report for Logistic Regression:              precision    recall  f1-score   support

           0       0.94      0.97      0.95       744
           1       0.76      0.63      0.69       131

    accuracy                           0.92       875
   macro avg       0.85      0.80      0.82       875
weighted avg       0.91      0.92      0.91       875



## Exp 2: Classification by training training Random Forest 

In [67]:
randomFrest_classifier = RandomForestClassifier(n_estimators=30, max_depth=3)
randomFrest_classifier.fit(x_train, y_train)
y_pred_ranFor = randomFrest_classifier.predict(x_test)
print("GK Classification report for Random Forest:" + classification_report(y_test, y_pred_ranFor))

GK Classification report for Random Forest:              precision    recall  f1-score   support

           0       0.96      0.99      0.97       744
           1       0.91      0.76      0.83       131

    accuracy                           0.95       875
   macro avg       0.93      0.87      0.90       875
weighted avg       0.95      0.95      0.95       875



### Exp 3: Classification by Training xgBoost

In [68]:
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(x_train, y_train)
y_pred_xgb = xgb_classifier.predict(x_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       744
           1       0.83      0.77      0.80       131

    accuracy                           0.94       875
   macro avg       0.90      0.87      0.88       875
weighted avg       0.94      0.94      0.94       875



In [69]:
!pip install imblearn

## Exp 4: Manage skewed classes using SMOTE and then training XGBoost

In [70]:
from imblearn.combine import SMOTETomek

smote = SMOTETomek(random_state=21)
x_train_res, y_train_res = smote.fit_resample(x_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([2152, 2152]))

In [71]:
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(x_train_res, y_train_res)

## Predict the text observations
y_pred_xgb = xgb_classifier.predict(x_test)
print("GK Classification report for smote and xgboost:" + classification_report(y_test, y_pred_xgb))

GK Classification report for smote and xgboost:              precision    recall  f1-score   support

           0       0.97      0.94      0.95       744
           1       0.70      0.84      0.76       131

    accuracy                           0.92       875
   macro avg       0.83      0.89      0.86       875
weighted avg       0.93      0.92      0.92       875



In [72]:

print("Shape of X_traing ------->>>>>>>>. X_traing.shape  ", X_train.shape)
print("Shape of Y_traing ------->>>>>>>>. Y_traing.shape  ", y_train.shape)
print("Shape of X_test ------->>>>>>>>. X_test.shape  ", X_test.shape)
print("Shape of Y_test ------->>>>>>>>. Y_test.shape  ", y_test.shape)

Shape of X_traing ------->>>>>>>>. X_traing.shape   (2625, 10)
Shape of Y_traing ------->>>>>>>>. Y_traing.shape   (2625,)
Shape of X_test ------->>>>>>>>. X_test.shape   (875, 10)
Shape of Y_test ------->>>>>>>>. Y_test.shape   (875,)


<h1 align="Left" style="color:Red">Below code Tracks the experiments using mlflow</h1>

In [73]:
models_dict = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (x_train_res, y_train_res),
        (x_test, y_test)
    )
]

In [78]:
reports = []

for this_model_name, this_model, train_set, test_set in models_dict:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    print("fitting the model ------->>>>>>>>. this_model_name  ", this_model_name)

    
    this_model.fit(X_train, y_train)
    y_pred = this_model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

fitting the model ------->>>>>>>>. this_model_name   Logistic Regression
fitting the model ------->>>>>>>>. this_model_name   Random Forest
fitting the model ------->>>>>>>>. this_model_name   XGBClassifier
fitting the model ------->>>>>>>>. this_model_name   XGBClassifier With SMOTE


In [79]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [80]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/21 16:00:55 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 16:00:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/843273881458092239/runs/2cd8ec5769d94748b2437472d14d46f8.
2024/08/21 16:00:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/843273881458092239.
2024/08/21 16:00:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 16:00:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/843273881458092239/runs/a2b0af618480433aa7b8d7b8cc6dee92.
2024/08/21 16:00:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/8432

NotFittedError: need to call fit or load_model beforehand